# Question 1
requirements
yfinance
numpy
pandas


In [2]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px
 
import time
from datetime import date


In [3]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
df = pd.read_html(url)[0]
df.loc[:, "Date added"] = pd.to_datetime(df["Date added"])
df.loc[:, "year"] = pd.to_datetime(df["Date added"]).dt.year
group = df.groupby("year").agg({
    "Symbol": lambda x: x.notna().sum()
})
group.sort_values("Symbol", ascending=False)
group.head()

,Symbol
year,
1957,53
1964,1
1965,2
1969,2
1970,2


In [4]:
df.loc[:, "year since"] = date.today().year -  pd.to_datetime(df["Date added"]).dt.year
filtered_df = df.loc[(df["year since"] > 20) & (df["year"] > 1957)].sort_values("year since", ascending=False)
print("How many companies are in the index for more than 20 year", len(filtered_df))
filtered_df.reset_index(inplace=True)
filtered_df

How many companies are in the index for more than 20 year 166


,index,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,year,year since
0,412,SHW,Sherwin-Williams,Materials,Specialty Chemicals,"Cleveland, Ohio",1964-06-30 00:00:00,89800,1866,1964,61
1,133,CMI,Cummins,Industrials,Construction Machinery & Heavy Transportation ...,"Columbus, Indiana",1965-03-31 00:00:00,26172,1919,1965,60
2,166,EMR,Emerson Electric,Industrials,Electrical Components & Equipment,"Ferguson, Missouri",1965-03-31 00:00:00,32604,1890,1965,60
3,109,CLX,Clorox,Consumer Staples,Household Products,"Oakland, California",1969-03-31 00:00:00,21076,1913,1969,56
4,334,NEM,Newmont,Materials,Gold,"Denver, Colorado",1969-06-30 00:00:00,1164727,1921,1969,56
...,...,...,...,...,...,...,...,...,...,...,...
161,281,LH,Labcorp,Health Care,Health Care Services,"Burlington, North Carolina",2004-11-01 00:00:00,920148,1978,2004,21
162,297,MTB,M&T Bank,Financials,Regional Banks,"Buffalo, New York",2004-02-23 00:00:00,36270,1856,2004,21
163,434,TPR,"Tapestry, Inc.",Consumer Discretionary,"Apparel, Accessories & Luxury Goods","New York City, New York",2004-09-01 00:00:00,1116132,2017,2004,21
164,444,TMO,Thermo Fisher Scientific,Health Care,Life Sciences Tools & Services,"Waltham, Massachusetts",2004-08-03 00:00:00,97745,2006 (1902),2004,21


In [5]:
# Load table
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
data = pd.read_html(url, header=[0, 1])[1]

# Safely assign datetime to the original DataFrame
date_col = pd.to_datetime(data[('Date', 'Date')])
data.loc[:, ('Date', 'Date')] = date_col

# Now extract year and compute year difference
data.loc[:, ('Date', 'year')] = date_col.dt.year
data.loc[:, ('Date', 'year since')] = date.today().year - date_col.dt.year

# Check result
filtered = data.loc[data[('Date', 'year since')] > 20]
filtered.sort_values(('Date', 'year since'), ascending=False)
print("how may companies are in the index more than 20 years: ", len(filtered))
filtered

how may companies are in the index more than 20 years:  18


Date  Added                                Removed  \
                    Date Ticker                       Security  Ticker   
354  2003-09-25 00:00:00   ESRX                Express Scripts    QTRN   
355  2000-12-05 00:00:00   INTU                         Intuit      BS   
356  2000-12-05 00:00:00    SBL            Symbol Technologies      OI   
357  2000-12-05 00:00:00    AYE               Allegheny Energy     GRA   
358  2000-12-05 00:00:00    ABK                Ambac Financial     CCK   
359  2000-07-27 00:00:00   JDSU                   JDS Uniphase     RAD   
360  2000-06-12 00:00:00    CVG                      Convergys     TMC   
361  2000-06-07 00:00:00   SBUX                      Starbucks     SMS   
362  1999-12-08 00:00:00   YHOO                         Yahoo!     LDW   
363  1999-06-09 00:00:00    WLP                      Wellpoint     HPH   
364  1999-04-12 00:00:00    ACT                        Actavis     NaN   
365  1998-12-11 00:00:00    FSR                        Firstar      AN   
366  1998-12-11 00:00:00    CCL                 Carnival Corp.     GRN   
367  1998-12-11 00:00:00   CPWR                      Compuware     SUN   
368  1997-06-17 00:00:00    CCI  Countrywide Credit Industries     USL   
369  1994-09-30 00:00:00    NCC                  National City     MCK   
370  1976-07-01 00:00:00    BUD                 Anheuser Busch     HNG   
371  1976-07-01 00:00:00    DIS        The Walt Disney Company     AYE   

                               \
                     Security   
354   Quintiles Transnational   
355           Bethlehem Steel   
356            Owens-Illinois   
357                  WR Grace   
358            Crown Holdings   
359                   RiteAid   
360              Times Mirror   
361    Shared Medical Systems   
362                   Laidlaw   
363  Harnischfeger Industries   
364                       NaN   
365                     Amoco   
366                General Re   
367                SunAmerica   
368                    USLife   
369                  McKesson   
370       Houston Natural Gas   
371          Allegheny Energy   

                                                Reason  Date             
                                                Reason  year year since  
354                                      Taken private  2003         22  
355                           Market Cap changes.[257]  2000         25  
356                                Market Cap changes.  2000         25  
357                                Market Cap changes.  2000         25  
358                                Market Cap changes.  2000         25  
359                            Market Cap change.[258]  2000         25  
360            Tribune Co. acquired Times Mirror.[259]  2000         25  
361   Siemens AG acquired Shared Medical Systems.[259]  2000         25  
362                 Market capitalization change.[260]  1999         26  
363           Harnischfeger filed for bankruptcy.[261]  1999         26  
364            Actavis plc (NYSE:ACT) added to S&P 500  1999         26  
365                           BP purchased Amoco.[262]  1998         27  
366      Berkshire Hathaway purchased General Re.[262]  1998         27  
367                     AIG purchased SunAmerica.[262]  1998         27  
368                          AIG acquired USLife.[263]  1997         28  
369  McKesson sold PCS Health Services to Eli Lilly...  1994         31  
370  Major restructuring of S&P 500 to have fewer i...  1976         49  
371  Major restructuring of S&P 500 to have fewer i...  1976         49

# Question 2


In [6]:
import datetime
end = date(2025, 5, 1) # As of May 1
start = date(2025, 1,1 )
print(f'Year = {end.year}; month= {end.month}; day={end.day}')
print(f'Period for indexes: {start} to {end} ')

def get_ytd_value(ticket:str, start:datetime.date=date(2025, 1, 1), end:datetime.date=date(2025,5 ,1 )):
    """return the YTD from a ticket from yahoo finance in %

    """

    df = yf.download(
        tickers = ticket,
        start=start, 
        end=end,
        interval = "1d")
    ytd = (df[( 'Close', ticket)].iloc[-1]/df[( 'Close', ticket)].iloc[0] -1)*100
    return float(ytd)


Year = 2025; month= 5; day=1
Period for indexes: 2025-01-01 to 2025-05-01 


In [7]:
countries = {
    "USA": "^GSPC",                         # S&P 500
    "China": "000001.SS",                   # Shanghai Composite
    "Hong Kong": "^HSI",                    # Hang Seng Index
    "Australia": "^AXJO",                   # S&P/ASX 200
    "India": "^NSEI",                       # Nifty 50
    "Canada": "^GSPTSE",                    # S&P/TSX Composite
    "Germany": "^GDAXI",                    # DAX
    "United Kingdom": "^FTSE",              # FTSE 100
    "Japan": "^N225",                       # Nikkei 225
    "Mexico": "^MXX",                       # IPC Mexico
    "Brazil": "^BVSP"                       # Ibovespa
}

data = []

for country, ticket in countries.items():
    ytd = get_ytd_value(ticket)
    data.append({
        "Country": country,
        "Ticket": ticket,
        "YTD": ytd
    })

df = pd.DataFrame(data)

# Get USA YTD for comparison
usa_ytd = df.loc[df["Country"] == "USA", "YTD"].values[0]

# Calculate difference from USA
df["S&P Diff"] = df["YTD"] - usa_ytd

# Optional: sort by difference
df = df.sort_values("S&P Diff", ascending=False)
df.reset_index(inplace=True)
del df["index"]

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
df

,Country,Ticket,YTD,S&P Diff
0,Mexico,^MXX,13.049444,18.152745
1,Hong Kong,^HSI,12.720018,17.823319
2,Brazil,^BVSP,12.438710,17.542010
3,Germany,^GDAXI,12.346378,17.449679
4,United Kingdom,^FTSE,2.842590,7.945891
5,India,^NSEI,2.490424,7.593724
6,China,000001.SS,0.504817,5.608118
7,Canada,^GSPTSE,-0.226126,4.877175
8,Australia,^AXJO,-0.914500,4.188800
9,USA,^GSPC,-5.103301,0.000000


# Question 3


    Download S&P 500 historical data (1950-present) using yfinance
    Identify all-time high points (where price exceeds all previous prices)
    For each pair of consecutive all-time highs, find the minimum price in between
    Calculate drawdown percentages: (high - low) / high × 100
    Filter for corrections with at least 5% drawdown
    Calculate the duration in days for each correction period
    Determine the 25th, 50th (median), and 75th percentiles for correction durations


In [9]:
df = yf.download(
    tickers = "^GSPC",
    start=date(1950, 1, 1), 
    end=date.today(),
    interval = "1d")["Close"]
df.loc[:, "cummax"] = df["^GSPC"].cummax()

[*********************100%***********************]  1 of 1 completed


In [10]:
# Prepare an empty DataFrame to collect corrections with >5% drawdown
corrections_df = pd.DataFrame()

for i in df["cummax"].unique():
    filtered_df = df[df["cummax"] == i].copy()

    filtered_df["diff"] = np.round((1 - filtered_df["^GSPC"] / filtered_df["cummax"]) * 100, 1)

    low_row = filtered_df.loc[filtered_df["^GSPC"].idxmin()] 
    min_price = low_row["^GSPC"]

    high_prices = filtered_df["cummax"]
    filtered_df["drawdown"] = np.round((high_prices - min_price) / high_prices * 100, 1)

    # Assuming filtered_df.index is a DatetimeIndex
    filtered_df['days_from_start'] = (filtered_df.index - filtered_df.index[0]).days
    date_started = filtered_df.index[0]


    # Get the first date where drawdown equals diff
    df_to_append = filtered_df[filtered_df["drawdown"] == filtered_df["diff"]].copy()
    df_to_append.loc[:, "Date_started"] = date_started
    df_to_append.index.rename("Date_ended", inplace=True)

    if df_to_append["drawdown"].iloc[0] > 5:
        corrections_df = pd.concat([corrections_df, df_to_append])

# Optional: reset index after concatenation
corrections_df = corrections_df.reset_index()
corrections_df  = corrections_df[[ 'Date_started', 'Date_ended', 'drawdown', 'days_from_start']]


In [11]:
corrections_df[corrections_df["Date_started"] == pd.Timestamp("1966-02-09")]

Ticker,Date_started,Date_ended,drawdown,days_from_start
20,1966-02-09,1966-10-07,22.2,240


In [12]:
corrections_df["days_from_start"].median()

np.float64(39.0)

# Question 4


In [16]:
url = "https://raw.githubusercontent.com/DataTalksClub/stock-markets-analytics-zoomcamp/refs/heads/main/cohorts/2025/ha1_Amazon.csv"
earnings_df = pd.read_csv(url, delimiter=';')
earnings_df.head(60)

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
0,AMZN,Amazon.com Inc,"April 29, 2026 at 6 AM EDT",-,-,-
1,AMZN,Amazon.com Inc,"February 4, 2026 at 4 PM EST",-,-,-
2,AMZN,Amazon.com Inc,"October 29, 2025 at 6 AM EDT",-,-,-
3,AMZN,Amazon.com Inc,"July 30, 2025 at 4 PM EDT",-,-,-
4,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",???.36,???.59,+16.74
5,AMZN,"Amazon.com, Inc.","February 6, 2025 at 4 PM EST",???.49,???.86,+24.47
6,AMZN,"Amazon.com, Inc.","October 31, 2024 at 4 PM EDT",???.14,???.43,+25.17
7,AMZN,"Amazon.com, Inc.","August 1, 2024 at 4 PM EDT",01.???,???.26,+22.58
8,AMZN,"Amazon.com, Inc.","April 30, 2024 at 4 PM EDT",0.83,0.98,+17.91
9,AMZN,"Amazon.com, Inc.","February 1, 2024 at 4 PM EST",0.8,1,+24.55


In [17]:
url = "https://raw.githubusercontent.com/DataTalksClub/stock-markets-analytics-zoomcamp/refs/heads/main/cohorts/2025/ha1_Amazon.csv"
earnings_df = pd.read_csv(url, delimiter=';')
# Convert date column to datetime
# Convert 'Earnings Date' to datetime
# Strip time zone info if parsing fails
earnings_df['Earnings Date'] = earnings_df['Earnings Date'].str.replace(r' [A-Z]{3}$', '', regex=True)
earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings Date'], format="%B %d, %Y at %I %p")


import re

def clean_eps(value):
    if pd.isna(value):
        return None
    # Remove non-numeric characters except period and minus sign
    cleaned = re.sub(r"[^0-9\.\-]", "", str(value))
    try:
        return float(cleaned)
    except ValueError:
        return None



# Rename for simplicity
earnings_df.rename(columns={
    'Earnings Date': 'date',
    'EPS Estimate': 'estimate',
    'Reported EPS': 'actual'
}, inplace=True)

earnings_df['estimate'] = earnings_df['estimate'].apply(clean_eps)
earnings_df['actual'] = earnings_df['actual'].apply(clean_eps)
earnings_df = earnings_df.dropna(subset=['estimate', 'actual'])
earnings_df

,Symbol,Company,date,estimate,actual,Surprise (%)
4,AMZN,"Amazon.com, Inc.",2025-05-01 16:00:00,0.36,0.59,+16.74
5,AMZN,"Amazon.com, Inc.",2025-02-06 16:00:00,0.49,0.86,+24.47
6,AMZN,"Amazon.com, Inc.",2024-10-31 16:00:00,0.14,0.43,+25.17
7,AMZN,"Amazon.com, Inc.",2024-08-01 16:00:00,1.00,0.26,+22.58
8,AMZN,"Amazon.com, Inc.",2024-04-30 16:00:00,0.83,0.98,+17.91
...,...,...,...,...,...,...
104,AMZN,"Amazon.com, Inc.",2000-04-26 00:00:00,-0.02,-0.02,+3.79
105,AMZN,"Amazon.com, Inc.",2000-02-02 00:00:00,-0.02,-0.03,-13.36
106,AMZN,"Amazon.com, Inc.",1999-10-27 00:00:00,-0.01,-0.01,+7.54
107,AMZN,"Amazon.com, Inc.",1999-07-21 00:00:00,-0.01,-0.01,+0.47


In [28]:
amazon = yf.download(
    tickers = "AMZN", 
    end=date.today(),
    interval = "1d")["Close"]
amazon['2d_return'] = np.round((amazon['AMZN'].shift(-1)/ amazon['AMZN'].shift(1)- 1)*100,1)
amazon.loc['2023-10-20':'2023-10-30']

[*********************100%***********************]  1 of 1 completed


Ticker,AMZN,2d_return
Date,,
2023-10-20,125.169998,-1.4
2023-10-23,126.559998,2.7
2023-10-24,128.559998,-4.1
2023-10-25,121.389999,-7.0
2023-10-26,119.570000,5.2
2023-10-27,127.739998,11.0
2023-10-30,132.710007,4.2


In [42]:
earnings_df = earnings_df[earnings_df['actual'] > earnings_df['estimate']].copy()
earnings_df = earnings_df.sort_values('date').reset_index(drop=True)
# Remove timezone from earnings dates (if present)
earnings_df['date'] = earnings_df['date'].dt.tz_localize(None)
earnings_df.date = earnings_df.date.dt.normalize()

print(f"Number of positive surprise dates: {len(earnings_df)}")
earnings_df

Number of positive surprise dates: 36


,Symbol,Company,date,estimate,actual,Surprise (%)
0,AMZN,"Amazon.com, Inc.",2006-02-02,0.01,0.02,+120.86
1,AMZN,"Amazon.com, Inc.",2008-07-23,0.01,0.02,+42.09
2,AMZN,"Amazon.com, Inc.",2009-01-29,0.02,0.03,+33.2
3,AMZN,"Amazon.com, Inc.",2010-10-21,0.02,0.03,+7.28
4,AMZN,"Amazon.com, Inc.",2011-01-27,0.04,0.05,+3.03
5,AMZN,"Amazon.com, Inc.",2012-01-31,0.01,0.02,127
6,AMZN,"Amazon.com, Inc.",2015-01-29,0.01,0.02,+168.18
7,AMZN,"Amazon.com, Inc.",2015-07-23,-0.01,0.01,+237.28
8,AMZN,"Amazon.com, Inc.",2015-10-22,-0.01,0.01,+225.37
9,AMZN,"Amazon.com, Inc.",2016-04-28,0.03,0.05,+81.42


In [43]:
# Merge to get 2-day return for earnings dates

merged = pd.merge(
    earnings_df,
    amazon[['2d_return']],
    left_on='date',
    right_index=True,
    how='left'
)
merged

,Symbol,Company,date,estimate,actual,Surprise (%),2d_return
0,AMZN,"Amazon.com, Inc.",2006-02-02,0.01,0.02,+120.86,-12.8
1,AMZN,"Amazon.com, Inc.",2008-07-23,0.01,0.02,+42.09,15.8
2,AMZN,"Amazon.com, Inc.",2009-01-29,0.02,0.03,+33.2,16.8
3,AMZN,"Amazon.com, Inc.",2010-10-21,0.02,0.03,+7.28,6.6
4,AMZN,"Amazon.com, Inc.",2011-01-27,0.04,0.05,+3.03,-2.4
5,AMZN,"Amazon.com, Inc.",2012-01-31,0.01,0.02,127,-6.6
6,AMZN,"Amazon.com, Inc.",2015-01-29,0.01,0.02,+168.18,16.7
7,AMZN,"Amazon.com, Inc.",2015-07-23,-0.01,0.01,+237.28,8.4
8,AMZN,"Amazon.com, Inc.",2015-10-22,-0.01,0.01,+225.37,7.8
9,AMZN,"Amazon.com, Inc.",2016-04-28,0.03,0.05,+81.42,8.7


In [ ]:
median_positive_surprise_return = merged['2d_return'].median()
print(f"Median 2-day return after positive surprises: {median_positive_surprise_return:.2f}%")

Median 2-day return after positive surprises: 2.65%


In [45]:
median_all_returns = amazon['2d_return'].median() 
print(f"Median 2-day return for all days: {median_all_returns:.2f}%")


Median 2-day return for all days: 0.20%
